### ***Logic***

##### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
##### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
     * Không quá 72h.
##### Backlog lấy
##### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
     * HCM -> HN: 12h.
     * Các tỉnh còn lại: 32h.
##### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
     * Giao hàng tại HCM HN: 36h.
     * Giao hàng tại các tỉnh còn lại: 56h.

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g
import os


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiKien',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu',
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(pd.to_datetime)

# set display of dataframe
pd.set_option('display.max_columns', 88)
pd.set_option('display.max_rows', 100)

# join export with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien']], on='MaDH', how='left')

# codes of e-commerces exchange
shopee_codes = ['18692']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery = data.loc[data['TrangThai'].isin([
    'Đang giao hàng', 'Giao không thành công',
    'Chờ xác nhận giao lại'
]) | ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] == data['KhoHienTai'])) |\
                          ((data['TrangThai'] == 'Tạo thành công') & ((~data['ThoiGianKetThucLay'].isnull()) ^\
                          (~data['ThoiGianKetThucGiao'].isnull())))]
delivery['LoaiBacklog'] = 'Xử lý giao'
internal_delivery = delivery['KhoLay'] == delivery['KhoHienTai']
delivery.loc[internal_delivery, 'N0'] = delivery.loc[internal_delivery]['ThoiGianKetThucLay']
delivery.loc[~internal_delivery, 'N0'] = delivery.loc[~internal_delivery]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(hours=120)
delivery['Aging'] = (datetime.today() - delivery['N+']).fillna(pd.Timedelta(hours=9999))


# return backlog (backlog trả)
returned = data.loc[data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn không thành công'
    ]) | ((data['TrangThai'] == 'Trả hàng') & (((~data['KhoTra'].isnull()) & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'].isnull()) & (data['KhoLay'] == data['KhoHienTai'])))) |\
          ((data['TrangThai'] == 'Tạo thành công') & (~data['ThoiGianKetThucGiao'].isnull()))]
returned['LoaiBacklog'] = 'Xử lý trả'
internal_return = returned['KhoHienTai'] == returned['KhoGiao']
returned.loc[internal_return, 'N0'] = returned.loc[internal_return]['ThoiGianKetThucGiao']
returned.loc[~internal_return, 'N0'] = returned.loc[~internal_return]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(hours=72)
returned['Aging'] = (datetime.today() - returned['N+']).fillna(pd.Timedelta(hours=9999))


# pickup backlog (backlog lấy)
pickup = data[(data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])) |\
              ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucLay'].isnull()) &\
               (data['KhoHienTai'] == data['KhoLay']))]
pickup['LoaiBacklog'] = 'Xử lý lấy'
ecoms_filter = pickup['MaKH'].isin(ecoms)
pickup.loc[ecoms_filter, 'N0'] = pickup.loc[ecoms_filter]['ThoiGianTaoChuyenDoi']
pickup.loc[~ecoms_filter, 'N0'] = pickup.loc[~ecoms_filter]['ThoiGianTao']
pickup['N+'] = pickup['N0'] + pd.Timedelta(hours=72)
pickup['Aging'] = (datetime.today() - pickup['N+']).fillna(pd.Timedelta(hours=9999))

# transporting backlog (backlog luân chuyển)
transporting = data.loc[(data['TrangThai'] == 'GHN đã lấy') |\
                         ((data['TrangThai'] == 'Lưu kho') & (data['KhoGiao'] != data['KhoHienTai']))]
transporting['LoaiBacklog'] = 'Xử lý luân chuyển'
transporting_filter = (transporting['DenTinh'] == 'Hà Nội') | (transporting['DenTinh'] == 'Hồ Chí Minh')
transporting['N0'] = transporting['ThoiGianKetThucLay']
transporting.loc[transporting_filter, 'N+'] = transporting.loc[transporting_filter]['N0']+ pd.Timedelta(hours=12)
transporting.loc[~transporting_filter, 'N+'] = transporting.loc[~transporting_filter]['N0'] + pd.Timedelta(hours=32)
transporting['Aging'] = (datetime.today() - transporting['N+']).fillna(pd.Timedelta(hours=9999))


# returned_transporting backlog (backlog luân chuyển trả hàng)
r_transporting = data.loc[data['TrangThai'].isin([
    'Trả hàng', 'Chờ trả hàng'
]) & (((data['KhoTra'].isnull()) & (data['KhoLay'] != data['KhoHienTai'])) |\
      ((~data['KhoTra'].isnull()) & (data['KhoTra'] != data['KhoHienTai'])))]
r_transporting['LoaiBacklog'] = 'Xử lý luân chuyển trả'
r_transporting_filter = (r_transporting['DenTinh'] == 'Hà Nội') | (r_transporting['DenTinh'] == 'Hồ Chí Minh')
r_transporting['N0'] = r_transporting['ThoiGianKetThucGiao']
r_transporting.loc[r_transporting_filter, 'N+'] = r_transporting.loc[r_transporting_filter]['N0'] + pd.Timedelta(hours=36)
r_transporting.loc[~r_transporting_filter, 'N+'] = r_transporting.loc[~r_transporting_filter]['N0'] + pd.Timedelta(hours=56)
r_transporting['Aging'] = (datetime.today() - r_transporting['N+']).fillna(pd.Timedelta(hours=9999))


# composite data
data = delivery.append([returned, pickup, transporting, r_transporting])
# convert Aging to hours
data['Aging (hours)'] = data['Aging']/np.timedelta64(1, 'h')
# set e-commerces column to NaN
data['e-commerces'] = np.nan


# name of e-commerces exchange
# shopee
data.loc[data['MaKH'].isin(shopee_codes), 'e-commerces'] = 'Shopee'
# sendo
data.loc[data['MaKH'].isin(sendo_codes), 'e-commerces'] = 'Sendo'
# tiki
data.loc[data['MaKH'].isin(tiki_codes), 'e-commerces'] = 'Tiki'
# lazada
data.loc[data['MaKH'].isin(lazada_codes), 'e-commerces'] = 'Lazada'


# data['ToanTrinh'] = data[data['MaKH'].isin(floors)]['N0'] - datetime.today()
# reset the index of dataframe
data.reset_index(inplace=True)
del data['index']
data['KhoGui'].astype('int64', errors='ignore')
data
# data['KhoGui']

,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,TenNguoiGui,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,LoaiBacklog,N0,N+,Aging,Aging (hours),e-commerces
0,FHPAFQS4,KV1293243226366,2020-08-01 10:57:01,2020-08-01 10:57:00,2020-08-01 16:10:36,2020-08-04 09:17:38,NaT,2020-08-04 23:00:00,NaT,NaN,Quận Hoàng Mai,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 25 Trương Định-Q.Hai Bà Trưng-HN 02,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,NaN,Đang giao hàng,500,1x1x1,518873,NaN,NaN,2 Ngày,1,1,0,0,1714228,9999,'ptlink',2267105,20NRU94012,2329.0,2275.0,2020-08-03 14:07:25,Xử lý giao,2020-08-03 14:07:25,2020-08-08 14:07:25,13 days 21:04:01.682356,333.067134,NaN
1,8FHRNA39X,20200809109882,2020-08-09 10:15:13,2020-08-09 10:15:13,2020-08-10 17:29:07,NaT,NaT,2020-08-12 10:01:00,NaT,NaN,Quận Bình Thạnh,Hồ Chí Minh,E,Quận 7,Hồ Chí Minh,E,Bưu Cục 219 Nơ Trang Long-Q.Bình Thạnh-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 219 Nơ Trang Long-Q.Bình Thạnh-HCM 01,Lưu kho,200,15x15x15,17578,'09/08/2020 09:14:15 - GHN-PFA1A0|THAY ĐỔI THÔ...,'',1 Ngày,2,0,0,NaN,1707089,NaN,'JUNO 246D xô viết Nghệ Tĩnh',1217523,20PVA93201,2329.0,2275.0,2020-08-11 02:56:37,Xử lý giao,2020-08-11 02:56:37,2020-08-16 02:56:37,6 days 08:14:49.682356,152.247134,NaN
2,8FHNL3F46,NaN,2020-08-04 15:42:45,2020-08-04 15:42:45,2020-08-04 16:26:44,2020-08-05 08:23:43,NaT,2020-08-08 23:00:00,NaT,NaN,Huyện Hớn Quản,Bình Phước,D,Quận 7,Hồ Chí Minh,E,Bưu Cục Chơn Thành-Bình Phước,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Chơn Thành-Bình Phước,Lưu kho,500,10x10x10,1045101,'05/08/2020 03:39:43 - GHN-DFC1A0|THAY ĐỔI THÔ...,'',2 Ngày,1,1,0,NaN,1883501,NaN,'St White',949270,20DMK99349,2329.0,2275.0,2020-08-05 03:12:50,Xử lý giao,2020-08-05 03:12:50,2020-08-10 03:12:50,12 days 07:58:36.682356,295.976856,NaN
3,8FH94YFU1,5f321a3826068e0001da0199_19,2020-08-11 11:14:53,2020-08-11 11:14:53,2020-08-11 19:25:43,2020-08-12 15:36:07,NaT,2020-08-15 23:00:00,NaT,NaN,Huyện Đức Hòa,Long An,F,Quận 4,Hồ Chí Minh,E,Bưu Cục Đức Hòa-Long An,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Đức Hòa-Long An,Lưu kho,500,10x10x10,1268922,'12/08/2020 09:43:59 - GHN-DFC1A4|KHÔNG LIÊN L...,'coban dau tieu xden 2x',2 Ngày,1,2,0,NaN,2033601,NaN,'hoàng đại',1168898,20FJX44012,2329.0,2275.0,2020-08-12 14:04:43,Xử lý giao,2020-08-12 14:04:43,2020-08-17 14:04:43,4 days 21:06:43.682356,117.112134,NaN
4,8FHSLFYHR,5f321c7226068e0001da01b3_7,2020-08-11 11:20:32,2020-08-11 11:20:32,2020-08-19 16:39:20,2020-08-20 10:13:49,NaT,2020-08-22 09:41:00,NaT,NaN,Quận 10,Hồ Chí Minh,E,Quận 4,Hồ Chí Minh,E,Bưu Cục 16A Lê Hồng Phong-Q.10-HCM 04,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 16A Lê Hồng Phong-Q.10-HCM 04,Lưu kho,500,10x10x10,189097,'11/08/2020 09:01:17 - GHN-PFA1A0|THAY ĐỔI THÔ...,'',1 Ngày,8,2,0,NaN,255990,NaN,'ĐẸP DESIGN',115282,20BJN25968,2329.0,2275.0,2020-08-19 23:47:50,Xử lý giao,2020-08-19 23:47:50,2020-08-24 23:47:50,-3 days +11:23:36.682356,-60.606477,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5827,FHFN31D1A,1000217031.#113728.1131884819.1023716211,2020-08-17 09:45:28,2020-08-17 09:45:28,2020-08-17 17:41:24,2020-08-18 07:33:48,2020-08-21 12:31:07,2020-08-19 23:00:00,NaT,'GHN-DCD0A4|KHÁCH TỪ CHỐI NHẬN/Hàng giao chậm ...,Quận Phú Nhuận,Hồ Chí Minh,E,Quận 7,Hồ Chí Minh,E,Bưu Cục 69 Hoàng Văn Thụ-Q.Phú Nhuận-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,NaN,Trả hàng,500,10x10x10,9315,'18/08/2020 12:13:47 - GHN-

In [5]:
# export data backlog to google sheets
# data export
data_compact = data[[
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'Aging (hours)',
    'e-commerces',
]]

# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog_everyday'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)
data_upload = data_compact.loc[data_compact['Aging'] >= pd.Timedelta(hours=0)]

# update data to google sheets
wks.clear()
df2g.upload(
    data_upload,
    SPREADSHEET_ID,
    WKS_NAME,
    credentials=credentials,
    row_names=False,
)


<Worksheet 'Backlog_everyday' id:181055436>

In [6]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)
# def iter_df(df):
#     """Generator for df."""
#     for val in df.columns:
#         yield val
#     for row in df.to_numpy():
#         for val in row:
#             if pd.isna(val):
#                 yield ''
#             else:
#                 yield val